In [1]:
import os
os.environ["R_HOME"] = "/home/huangzhuoli/mambaforge/envs/tensorqtl/lib/R"
os.environ["R_LIBS_USER"] = "/home/huangzhuoli/mambaforge/envs/tensorqtl/lib/R/library"
import pandas as pd
import tensorqtl

In [2]:
from tensorqtl import rfunc

In [ ]:
#处理eQTL

In [ ]:
eqtl_celltype = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scRNA.csv')
eqtl_celltype = eqtl_celltype ['final_annotation']

In [4]:
trans_df_all = pd.DataFrame()
for celltype in eqtl_celltype:
    print(f'reading_{celltype}')
    trans_df = pd.read_parquet(f'/CIMA/Result/eQTL_trans/{celltype}.parquet')
    trans_df_all = pd.concat([trans_df_all, trans_df], axis=0, ignore_index=True) 

reading_Atypical_Bm_ITGAX
reading_Bn_IFIT3
reading_Bn_TCL1A
reading_Bn_IL6
reading_CD4_CTL_GZMH
reading_CD4_Tcm_CXCR5
reading_CD4_Tcm_IFI44L
reading_CD4_Tem_CCR7neg
reading_CD4_Tfh-like_CXCR5
reading_CD4_Th1-like_GZMK
reading_CD4_Th17-like_RORC
reading_CD4_Th22-like_CCR10
reading_CD4_Th_CCR4
reading_CD4_Th_CR1
reading_CD4_Th_LMNA
reading_CD4_Th_TNFRSF11A
reading_CD4_Tn_CCR7
reading_CD4_Tn_CXCR5
reading_CD4_Tn_LIMS1
reading_CD4_Tn_SOX4
reading_CD4_Tr1-like_IL10
reading_CD4_Treg_FCRL3
reading_CD4_Treg_FOXP3
reading_CD8_CTL_GZMB
reading_CD8_CTL_GZMK
reading_CD8_CTL_IFI44L
reading_CD8_Tcm_IFI44L
reading_CD8_Tem_CCR7neg
reading_CD8_Tn_CCR7
reading_CD8_Tn_SOX4
reading_Cycling_NK_MKI67
reading_Cycling_T_MKI67
reading_pre-T-like_CABP4
reading_HSPC_CD34
reading_ILC2_IL2RA
reading_MAIT_SLC4A10
reading_MK_GP9
reading_NKT_NCR1
reading_NKT_IFNG
reading_NK_bright_XCL1
reading_Plasma_IGHA1
reading_Plasma_IGHG1
reading_Plasmablast_MKI67
reading_Switched_Bm_IGHE
reading_Switched_activated_Bm_CD86
readi

In [6]:
trans_df_all.to_parquet('/CIMA/Result/summary/20250110_trans_eQTL_full.parquet')

In [ ]:
#筛选下游的结果

In [ ]:
trans_df_all = pd.read_parquet('/CIMA/Result/summary/20250110_trans_eQTL_full.parquet')

#去除同一条染色体上的结果
trans_df_all = trans_df_all[trans_df_all['variant_chr'] != trans_df_all['phenotype_chr']]

trans_df_all.loc[:,'pair'] = trans_df_all['variant_id'].astype(str) + '_' + trans_df_all['phenotype_id'].astype(str)

In [ ]:
#做BH
import pandas as pd
from statsmodels.stats.multitest import multipletests
trans_df_all.loc[:,'padj'] = multipletests(trans_df_all['pval'], method='fdr_bh')[1]
trans_df_all_sig = trans_df_all[trans_df_all['padj'] < 0.05].reset_index(drop = True)
trans_df_all_sig['variant_celltype'] = trans_df_all_sig['variant_id']+"_"+trans_df_all_sig['celltype']

In [ ]:
#合并cis-eQTL的结果
lead_esnp_all_df =  pd.read_csv('/CIMA/Result/20250108_cis_eQTL_studywise_sig.csv',index_col=0) 

lead_esnp_all_df['variant_celltype'] = lead_esnp_all_df['variant_id']+"_"+lead_esnp_all_df['celltype']

lead_snp_egene = lead_esnp_all_df.groupby('variant_celltype')['phenotype_id'].agg(';'.join).reset_index()

lead_snp_egene = lead_snp_egene.rename(columns={'phenotype_id': 'cis_eGene'})

trans_df_all_sig_merge = pd.merge(trans_df_all_sig, lead_snp_egene, on='variant_celltype', how='inner')

trans_df_all_sig_merge['trans_eGene'] = trans_df_all_sig_merge['phenotype_id']

trans_df_all_sig_merge.to_csv('/CIMA/Result/summary/20250110_trans_eQTL_fdr005.csv')

In [3]:
#处理caQTL

In [ ]:
caqtl_celltype = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scATAC.csv')
caqtl_celltype = caqtl_celltype ['final_annotation']

In [6]:
#trans_df_all = pd.DataFrame()
#for celltype in caqtl_celltype:
#    print(f'reading_{celltype}')
#    trans_df = pd.read_parquet(f'/CIMA/Result/caQTL_trans/{celltype}.parquet')
#    trans_df_all = pd.concat([trans_df_all, trans_df], axis=0, ignore_index=True) 

In [5]:
# 使用列表存储所有 DataFrame
trans_df_list = [
    pd.read_parquet(f'/CIMA/Result/caQTL_trans/{celltype}.parquet')
    for celltype in caqtl_celltype
]

# 一次性合并
trans_df_all = pd.concat(trans_df_list, axis=0, ignore_index=True)

In [6]:
del trans_df_list
import gc
gc.collect()

60

In [7]:
trans_df_all

,variant_id,phenotype_id,pval,b,b_se,af,variant_chr,phenotype_chr,celltype
0,chr1_893503,chr1:629700-630201,4.977980e-01,-0.071484,0.105304,0.209231,1,1,Atypical_Bm_ITGAX
1,chr1_893503,chr1:633771-634272,5.449140e-01,0.064975,0.107196,0.209231,1,1,Atypical_Bm_ITGAX
2,chr1_893503,chr1:778491-778992,1.878104e-10,0.633154,0.095762,0.209231,1,1,Atypical_Bm_ITGAX
3,chr1_893503,chr1:827286-827787,4.405727e-01,0.071233,0.092234,0.209231,1,1,Atypical_Bm_ITGAX
4,chr1_893503,chr1:865547-866048,9.218741e-02,0.151274,0.089528,0.209231,1,1,Atypical_Bm_ITGAX
...,...,...,...,...,...,...,...,...,...
7122724589,chr22_50525807,chr22:50469787-50470288,6.399828e-01,-0.051792,0.110517,0.227979,22,22,pre-Switched_Bm_JAM3
7122724590,chr22_50525807,chr22:50530801-50531302,1.583826e-01,-0.141391,0.099762,0.227979,22,22,pre-Switched_Bm_JAM3
7122724591,chr22_50525807,chr22:50539528-50540029,7.634677e-01,0.031535,0.104610,0.227979,22,22,pre-Switched_Bm_JAM3
7122724592,chr22_50525807,chr22:50582746-50583247,5.517996e-06,-0.487746,0.103653,0.227979,22,22,pre-Switched_Bm_JAM3


In [8]:
trans_df_all['celltype'].value_counts()

celltype
cMono_CD14                    3164646576
CD4_Tn_CCR7                    449855420
ncMono_FCGR3A                  442648965
Mature_NK_dim_FCGR3A           387503160
cDC2_CD1C                      387286437
CD8_Tem_CCR7neg                302257824
CD8_CTL_GZMB                   255863520
NKT_NCR1                       253687126
CD8_Tn_CCR7                    214078807
MAIT_SLC4A10                   187477642
Bn_TCL1A                       184338809
CD4_Th1-like_GZMK              180301761
cMono_IFI44L                   100216500
CD4_Tfh-like_CXCR5              74770227
intMono_GFRA2                   74636910
gdT2_IL12RB2                    68462520
Switched_Bm_IGHDneg             59299980
ncMono_C1QA                     53402559
CD4_Tem_CCR7neg                 52169440
gdT2_GZMH                       35572583
pDC_IRF4                        32395234
CD4_Th17-like_RORC              26653085
Transitional_B_SOX4             22727232
CD4_CTL_GZMH                    20884160
Transit

In [9]:
trans_df_all.to_parquet('/CIMA/CIMA_r1/test/20250110_trans_caQTL_full.parquet')

In [3]:
trans_df_all = pd.read_parquet('/CIMA/CIMA_r1/test/20250110_trans_caQTL_full.parquet')

In [10]:
#去除同一条染色体上的结果
trans_df_all = trans_df_all.query('variant_chr != phenotype_chr')
print('done')
# 操作和 pandas 基本相同
trans_df_all['pair'] = trans_df_all['variant_id'].astype(str) + '_' + trans_df_all['phenotype_id'].astype(str)
print('done')

done
done


/tmp/ipykernel_2476465/3940533042.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_df_all['pair'] = trans_df_all['variant_id'].astype(str) + '_' + trans_df_all['phenotype_id'].astype(str)


In [11]:
import pandas as pd
from statsmodels.stats.multitest import multipletests

# 提取 p 值为 NumPy 数组
pvals = trans_df_all['pval'].values
print('done')
# 执行 Benjamini-Hochberg 校正
_, padj, _, _ = multipletests(pvals, method='fdr_bh')
print('done')
# 将结果赋值回 DataFrame
trans_df_all['padj'] = padj
print('done')
# 筛选 padj < 0.05 的显著结果
trans_df_all_sig = trans_df_all[trans_df_all['padj'] < 0.05].reset_index(drop=True)
print('done')
# 创建新列 'variant_celltype'
trans_df_all_sig['variant_celltype'] = trans_df_all_sig['variant_id'] + "_" + trans_df_all_sig['celltype']
print('done')

done
done


/tmp/ipykernel_2476465/1565689682.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_df_all['padj'] = padj


done
done
done


In [30]:
trans_df_all.to_parquet('/CIMA/CIMA_r1/test/20250110_trans_caQTL_with_FDR.parquet')

In [13]:
trans_df_all_sig.to_csv('/CIMA/Result/summary/20250114_trans_caQTL_fdr005.csv')

In [14]:
#合并cis-caQTL的结果
lead_esnp_all_df =  pd.read_csv('/CIMA/Result/20250108_cis_caQTL_studywise_sig.csv',index_col=0) 

lead_esnp_all_df['variant_celltype'] = lead_esnp_all_df['variant_id']+"_"+lead_esnp_all_df['celltype']

lead_snp_egene = lead_esnp_all_df.groupby('variant_celltype')['phenotype_id'].agg(';'.join).reset_index()

lead_snp_egene = lead_snp_egene.rename(columns={'phenotype_id': 'cis_caPeak'})

trans_df_all_sig_merge = pd.merge(trans_df_all_sig, lead_snp_egene, on='variant_celltype', how='inner')

trans_df_all_sig_merge['trans_caPeak'] = trans_df_all_sig_merge['phenotype_id']


In [15]:
trans_df_all_sig_merge.to_csv('/CIMA/Result/summary/20250114_trans_caQTL_fdr005.csv')

In [109]:
#查看有无位点既有cis-eQTL/caQTL效应也有trans-eQTL/caQTL效应

In [17]:
trans_df_all_sig_merge = pd.read_csv('/CIMA/Result/summary/20250114_trans_caQTL_fdr005.csv',index_col=0)

In [18]:
trans_df_all_sig_merge_eQTL =  pd.read_csv('/CIMA/Result/summary/20250110_trans_eQTL_fdr005.csv',index_col=0)

In [19]:
trans_df_all_sig_merge_eQTL = trans_df_all_sig_merge_eQTL[
    trans_df_all_sig_merge_eQTL['variant_celltype'].isin(trans_df_all_sig_merge['variant_celltype'])
]

In [20]:
trans_df_all_sig_merge = trans_df_all_sig_merge.loc[:,["variant_celltype","cis_caPeak","trans_caPeak"]]

In [21]:
trans_df_all_sig_merge_e_caQTL = pd.merge(trans_df_all_sig_merge_eQTL, trans_df_all_sig_merge, on='variant_celltype', how='inner')

In [22]:
trans_df_all_sig_merge_eQTL['variant_celltype'].unique()

array(['chr6_130185237_CD8_Tn_CCR7', 'chr17_40096407_CD8_Tn_CCR7',
       'chr9_136619405_Mature_NK_dim_FCGR3A',
       'chr9_133003751_ncMono_FCGR3A'], dtype=object)

In [23]:
trans_df_all_sig_merge_e_caQTL['variant_celltype'].unique()

array(['chr6_130185237_CD8_Tn_CCR7', 'chr17_40096407_CD8_Tn_CCR7',
       'chr9_136619405_Mature_NK_dim_FCGR3A',
       'chr9_133003751_ncMono_FCGR3A'], dtype=object)

In [24]:
# 提取指定列
selected_columns = ['variant_celltype', 'cis_eGene', 'trans_eGene', 'cis_caPeak', 'trans_caPeak']
trans_df_all_sig_merge_e_caQTL = trans_df_all_sig_merge_e_caQTL[selected_columns]

In [25]:
trans_df_all_sig_merge_e_caQTL = trans_df_all_sig_merge_e_caQTL.groupby('variant_celltype').agg({
    'cis_eGene': lambda x: ';'.join(x.astype(str)),
    'trans_eGene': lambda x: ';'.join(x.astype(str)),
    'cis_caPeak': lambda x: ';'.join(x.astype(str)),
    'trans_caPeak': lambda x: ';'.join(x.astype(str))
}).reset_index()

In [26]:
trans_df_all_sig_merge_e_caQTL

,variant_celltype,cis_eGene,trans_eGene,cis_caPeak,trans_caPeak
0,chr17_40096407_CD8_Tn_CCR7,THRA,ARNTL,chr17:40068745-40069246;chr17:40069867-4007036...,chr10:63421561-63422062
1,chr6_130185237_CD8_Tn_CCR7,AL355581.1;SAMD3;AL355581.1;SAMD3,DGKD;DGKD,chr6:130183864-130184365;chr6:130183864-130184365,chr2:233419586-233420087;chr8:124637460-124637961
2,chr9_133003751_ncMono_FCGR3A,GFI1B;GFI1B;GFI1B;GFI1B;GFI1B;GFI1B;GFI1B;GFI1...,RBP7;RBP7;RBP7;RBP7;RBP7;RBP7;RBP7;CLEC4A;CLEC...,chr9:132977604-132978105;chr9:132978468-132978...,chr1:27099453-27099954;chr1:85278804-85279305;...
3,chr9_136619405_Mature_NK_dim_FCGR3A,NALT1;NALT1;NALT1;NALT1;NALT1;NALT1;NALT1;NALT...,JAK1;JAK1;JAK1;JAK1;DENND2D;DENND2D;DENND2D;DE...,chr9:136545948-136546449;chr9:136563049-136563...,chr1:999918-1000419;chr14:68711726-68712227;ch...


In [27]:
# 去除每个列中以 ";" 分隔的子元素的重复值
def remove_duplicates(x):
    # 将字符串分割为列表，去除重复值，再用 ";" 连接
    return ';'.join(sorted(set(x.split(';')), key=x.split(';').index))

# 对每一列应用去重操作
trans_df_all_sig_merge_e_caQTL[['cis_eGene', 'trans_eGene', 'cis_caPeak', 'trans_caPeak']] = trans_df_all_sig_merge_e_caQTL[['cis_eGene', 'trans_eGene', 'cis_caPeak', 'trans_caPeak']].map(remove_duplicates)

In [28]:
trans_df_all_sig_merge_e_caQTL.to_csv('/CIMA/Result/summary/20250114_trans_e_caQTL_overlap.csv')

In [29]:
trans_df_all_sig_merge_e_caQTL

,variant_celltype,cis_eGene,trans_eGene,cis_caPeak,trans_caPeak
0,chr17_40096407_CD8_Tn_CCR7,THRA,ARNTL,chr17:40068745-40069246;chr17:40069867-4007036...,chr10:63421561-63422062
1,chr6_130185237_CD8_Tn_CCR7,AL355581.1;SAMD3,DGKD,chr6:130183864-130184365,chr2:233419586-233420087;chr8:124637460-124637961
2,chr9_133003751_ncMono_FCGR3A,GFI1B,RBP7;CLEC4A;LRP1;USP36;KIAA0355,chr9:132977604-132978105;chr9:132978468-132978...,chr1:27099453-27099954;chr1:85278804-85279305;...
3,chr9_136619405_Mature_NK_dim_FCGR3A,NALT1,JAK1;DENND2D;CD244;PPM1L;SPON2;GSAP;PTPN12;PRF...,chr9:136545948-136546449;chr9:136563049-136563...,chr1:999918-1000419;chr14:68711726-68712227;ch...
